In [1]:
import torch
import sys,os
sys.path.append(os.pardir)
from bert_seq2seq.tokenizer import Tokenizer, load_chinese_base_vocab
import json
import time
import bert_seq2seq
from bert_seq2seq.utils import load_bert
import lawrouge

In [2]:
model = "./model_file/trained_model/summary_roberta_wwm.bin"
# model2 = "./model_file/trained_model/summary_roberta_wwm_256.bin"
vocab_text = "./vocab/vocab.txt"
model_name = "roberta"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 加载字典
word2idx,_ = load_chinese_base_vocab(vocab_path=vocab_text,simplfied=True)
tokenizer = Tokenizer(word2idx)
# 定义模型
bert_model = load_bert(word2idx,model_name=model_name)
bert_model.set_device(device)
bert_model.eval()
bert_model.load_all_params(model_path=model, device=device)

精简后的词表大小为：13584
./model_file/trained_model/summary_roberta_wwm.bin loaded!


In [3]:
import pandas as pd
df = pd.read_csv("./dataset/evaluation_with_ground_truth.csv")
del df["Unnamed: 0"]
df.head()

,summary,article
0,北京精神病人被用束缚带绑床上，遭病友掐死；法院认定医院管理不当、值班护士失职，构成医疗事故罪。\n,24岁的男子小罗（化名）因患有精神病被送进医院，为防止他伤害自己，医院用保护带将其绑在床...
1,世界卫生组织发布报告称，在过去10年，自杀取代难产死亡，成为全球年轻女性死亡的最主要原因,中新网5月26日电据外媒报道，世界卫生组织日前发布了一份报告，指出自杀已经取代难产，成为...
2,温州瑞安重度烧伤孕妇已有清晰意识，急需A型血小板；5月9日其煮夜宵时发生爆燃，已收捐款超600万。,温州网讯昨天，广受关注的“瑞安孕妇重度烧伤”一事有了新进展：家属称，王芙蓉现阶段治疗急需...
3,据雅虎体育，莫里斯-威廉姆斯与骑士达成协议，将与詹皇重聚，他将签下2年430万美元的合同,2015-07-0710:00新浪体育大神爆料北京时间7月7日消息，据《雅虎体育》报道，...
4,天津：5月31日起外地人买房不再送户口，取消蓝印户口政策，以“积分落户”取代。,京华时报记者张然商西邢飞天津将于5月底取消蓝印户口政策，“积分落户”将取而代之，这意味着...


In [27]:
test_data = df["article"][:100]
df["summary"][:100].to_csv("test.txt")


In [28]:
res = []
for text in test_data:
    with torch.no_grad():
        temp = bert_model.generate(text, beam_size=4,out_max_length=150)
        res.append(temp.replace(" ",""))

In [29]:
res_df = pd.DataFrame(res)
res_df.head()
res_df.to_csv("res.txt")

In [53]:
rouge = lawrouge.Rouge()
score = rouge.get_scores('./res.txt','./test.txt',avg=0)
score

[{'rouge-1': {'f': 0.8421052581717452, 'p': 0.8888888888888888, 'r': 0.8},
  'rouge-2': {'f': 0.5882352891349482, 'p': 0.625, 'r': 0.5555555555555556},
  'rouge-l': {'f': 0.8421052581717452, 'p': 0.8888888888888888, 'r': 0.8}}]

In [42]:
score4 = rouge.get_scores(res[4], test_res[4], avg=0)
score4

[{'rouge-1': {'f': 0.459999995242,
   'p': 0.3770491803278688,
   'r': 0.5897435897435898},
  'rouge-2': {'f': 0.2857142809662641,
   'p': 0.23333333333333334,
   'r': 0.3684210526315789},
  'rouge-l': {'f': 0.3999999952420001,
   'p': 0.32786885245901637,
   'r': 0.5128205128205128}}]

In [38]:
score

{'f': 0.7405572705570265, 'p': 0.7833333333333334, 'r': 0.7022222222222223}

In [33]:
score2

[{'rouge-1': {'f': 0.6506024046566993,
   'p': 0.6923076923076923,
   'r': 0.6136363636363636},
  'rouge-2': {'f': 0.49382715551287915,
   'p': 0.5263157894736842,
   'r': 0.46511627906976744},
  'rouge-l': {'f': 0.6506024046566993,
   'p': 0.6923076923076923,
   'r': 0.6136363636363636}}]

In [15]:
score3

{'f': 0.5951219462468241, 'p': 0.6425320056899004, 'r': 0.5542283298097253}

In [13]:
res[1],test_res[1]

('世界卫生组织发布了一份报告，指出自杀已经取代难产，成为全球年轻女性的头号杀手。',
 '世界卫生组织发布报告称，在过去10年，自杀取代难产死亡，成为全球年轻女性死亡的最主要原因')